<a href="https://colab.research.google.com/github/hectic97/Boostcamp-AI-Tech/blob/main/U-stage/Week8/parallel_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install ray
!pip install ray

     |████████████████████████████████| 47.5MB 91kB/s 
     |████████████████████████████████| 1.3MB 41.2MB/s 
     |████████████████████████████████| 3.2MB 27.2MB/s 
     |████████████████████████████████| 204kB 44.3MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 133kB 35.9MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 296kB 47.6MB/s 
     |████████████████████████████████| 143kB 55.6MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=ffe3e4d4d329691744335d2ecbd8c887cd2dab7d654910a82f74cdedc7e6e76d
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat


In [3]:
import ray
import time

# Check node
ray.init()

2021-03-17 01:11:47,706	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


{'metrics_export_port': 57166,
 'node_id': '9acaf34c33c69eb046e5386b020b58c2f62dc4eb8caaf925bdd79fa8',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-03-17_01-11-47_007509_62/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-03-17_01-11-47_007509_62/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-03-17_01-11-47_007509_62',
 'webui_url': '127.0.0.1:8265'}

## ray

remote function은 Ray의 프로세스에 의해 비동기적으로 실행됩니다. 따라서 아래의 코드는 ray에 의해서 비동기적으로 실행됩니다.

In [7]:
@ray.remote(num_cpus=1)
def f(x):
    time.sleep(1)
    return x

def g(x):
    time.sleep(1)
    return x

In [9]:
start = time.time()
# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))

# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

print("Ray 소요시간: {}".format(time.time() - start))

start = time.time()
result_ids = []
for i in range(4):
    result_ids.append(g(i))
print("No Ray 소요시간: {}".format(time.time()-start))

Ray 소요시간: 2.0075857639312744
No Ray 소요시간: 4.003834247589111


## Sequential

## Task Dependency

한 Task가 끝나야 다른 Task를 실행할 수 있는 경우가 있습니다. 이런 경우를 Task Dependency가 있다고 합니다.

병렬로 x_id와 y_id를 연산하고 z_id를 구하는 예제

```
[x_id, y_id] > z_id
```

In [15]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

size = 0 # check O(?) as # of input increases

# Get the results with ray.
start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("{} z_id_with_ray : {}".format(size, time.time() - start))

size = 10

start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("{} z_id_with_ray : {}".format(size,time.time() - start))

size = 100

start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("{} z_id_with_ray : {}".format(size,time.time() - start))

size = 1000

start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("{} z_id_with_ray : {}".format(size,time.time() - start))



0 z_id_with_ray : 0.03999590873718262
10 z_id_with_ray : 0.015011310577392578
100 z_id_with_ray : 0.019176006317138672
1000 z_id_with_ray : 0.3899657726287842


## Aggregation

연산을 어떻게 병렬로 배치하느냐에 따라서 보는 속도의 차이
![](https://miro.medium.com/max/1400/1*vHz3troEmr4uLns0V8VmdA.jpeg)


In [16]:
import time

@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

In [17]:
start = time.time()
# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)
print("Vanilla version : {}".format(time.time() - start))


start = time.time()
# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)
print("Advanced version : {}".format(time.time() - start))

Vanilla version : 7.0391762256622314
Advanced version : 4.01997971534729


## Reference

https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8